## 🎓 𝗗𝗦𝗣: 𝗗emonstrate–𝗦earch–𝗣redict

This notebook illustrates the Demonstrate–Search–Predict (𝗗𝗦𝗣) framework on the multi-hop question answering (QA) task.

Below, we'll build **five** small 𝗗𝗦𝗣 programs that implement strategies for this task. The programs range from extremely simple ones (programs 1 and 2) to moderate ones (programs 3 and 4). Program 5 considers a fairly advanced pipeline.

### Installation

If you haven't installed **DSP** already, let's do that.

In [ ]:
!pip install dsp-ml

try:
    import google.colab  # When on google Colab, let's clone the notebook so we download the cache.
    !git -C dsp/ pull || git clone https://github.com/stanfordnlp/dsp
    %set_env DSP_NOTEBOOK_CACHEDIR dsp/cache

except:
    %set_env DSP_NOTEBOOK_CACHEDIR cache  # Set up the cache for the examples below.

### Setting Up

We'll start by setting up the language model (LM) and retrieval model (RM).

We will work with the **GPT-3.5** LM (`text-davinci-002`) and the **ColBERTv2** RM.

To use GPT-3, you'll need an OpenAI key. For ColBERTv2, we've set up a server hosting a Wikipedia (Dec 2018) search index, so you don't need to worry about setting one up!

To make things easy, we've set up a cache in this repository. _If you want to run this notebook without changing the code or examples, you don't need an API key. All examples are cached._

In [2]:
%load_ext autoreload
%autoreload 2

import os
import dsp

openai_key = os.getenv('OPENAI_API_KEY')  # or replace with your API key (optional)
colbert_server = 'http://ec2-44-228-128-229.us-west-2.compute.amazonaws.com:8893/api/search'

lm = dsp.GPT3(model='text-davinci-002', api_key=openai_key)
rm = dsp.ColBERTv2(url=colbert_server)

dsp.settings.configure(lm=lm, rm=rm)

### Task Examples

Next, let's look at a few examples of the task. Each example consists of a question and one or more gold answers.

We have six training examples (`train`), which we'll feed into the programs. These will help define the task.

Notice that our examples only have input (`question`) and output (`answer`) fields. When our advanced programs build sophisticated pipelines, training "demonstrations" for other fields will be constructed automatically.

In [3]:
train = [('Who produced the album that included a re-recording of "Lithium"?', ['Butch Vig']),
         ('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', ['Kevin Greutert']),
         ('The heir to the Du Pont family fortune sponsored what wrestling team?', ['Foxcatcher', 'Team Foxcatcher', 'Foxcatcher Team']),
         ('In what year was the star of To Hell and Back born?', ['1925']),
         ('Which award did the first book of Gary Zukav receive?', ['U.S. National Book Award', 'National Book Award']),
         ('What city was the victim of Joseph Druces working in?', ['Boston, Massachusetts', 'Boston']),]

train = [dsp.Example(question=question, answer=answer) for question, answer in train]

The development examples (`dev`) will be used to assess the behavior of each program we build. Of course, this tiny set is not meant to be a reliable benchmark, but it'll be instructive to use it for illustration.

In [4]:
dev = [('Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?', ['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']),
       ('What documentary about the Gilgo Beach Killer debuted on A&E?', ['The Killing Season']),
       ('Right Back At It Again contains lyrics co-written by the singer born in what city?', ['Gainesville, Florida', 'Gainesville']),
       ('What year was the party of the winner of the 1971 San Francisco mayoral election founded?', ['1828']),
       ('Which author is English: John Braine or Studs Terkel?', ['John Braine']),
       ('Anthony Dirrell is the brother of which super middleweight title holder?', ['Andre Dirrell']),
       ('In which city is the sports nutrition business established by Oliver Cookson based ?', ['Cheshire', 'Cheshire, UK']),
       ('Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.', ['February 13, 1980']),
       ('Kyle Moran was born in the town on what river?', ['Castletown', 'Castletown River']),
       ("What is the name of one branch of Robert D. Braun's speciality?", ['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']),
       ("Where was the actress who played the niece in the Priest film born?", ['Surrey', 'Guildford, Surrey']),
       ('Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.', ['Portrait of a Marriage']),
       ('What year was the father of the Princes in the Tower born?', ['1442'])]

dev = [dsp.Example(question=question, answer=answer) for question, answer in dev]

### Program 1: Vanilla GPT-3.5

Let's begin by implementing the simplest program for this task. We'll prompt **GPT-3.5** to answer each question based on its internal parameteric knowledge.

We'll start by defining the `Template` that defines how we will communicate with the LM.

Specifically, the question–answer template (`qa_template`) will include a question and a short answer for each example.

In [5]:
Question = dsp.Type(prefix="Question:", desc="${the question to be answered}")
Answer = dsp.dsp.Type(prefix="Final Answer:", desc="${a short factoid answer, often between 1 and 5 words}", format=dsp.format_answers)

qa_template = dsp.Template(instructions="Answer questions with short factoid answers.", question=Question(), answer=Answer())

Then, let's define the actual program, `vanilla_LM_QA`. It'll accept a string (`question`) and returns another string (its short `answer`).

In [6]:
def vanilla_LM_QA(question: str) -> str:
    demos = dsp.sample(train, k=7)
    example = dsp.Example(question=question, demos=demos)

    example, completions = dsp.generate(qa_template)(example, stage='qa')
    return completions.answer

Let's invoke the program on a sample question.

In [7]:
print(dev[0].question)
print(vanilla_LM_QA(dev[0].question))

Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
E. L. Doctorow


Let's inspect the last call to the LM to learn more about the behavior of the program.

In [8]:
lm.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${the question to be answered}
Final Answer: ${a short factoid answer, often between 1 and 5 words}

---

Question: Which award did the first book of Gary Zukav receive?
Final Answer: U.S. National Book Award

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Final Answer: Foxcatcher

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Final Answer: Kevin Greutert

Question: Who produced the album that included a re-recording of "Lithium"?
Final Answer: Butch Vig

Question: What city was the victim of Joseph Druces working in?
Final Answer: Boston, Massachusetts

Question: In what year was the star of To Hell and Back born?
Final Answer: 1925

Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Final Answer: E. L. Doctorow





We can see that the program has organized the examples in a simple prompt and generated an answer for the `dev[0]` question we asked.

Let's try on the remaining development examples.

In [9]:
from dsp.evaluation.utils import evaluate

evaluate(vanilla_LM_QA, dev)

100%|██████████| 13/13 [00:00<00:00, 384.43it/s]

Answered 4 / 13 correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Melbourne, Australia",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1966,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Manchester, England",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","July 3, 1961",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Susquehanna River,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Mars exploration,❌


### Program 2: Retrieve-then-Read w/ GPT-3.5

The program above relies entirely on the factual knowledge memorized by GPT-3. As we have seen, it's sometimes correct but it's far from reliable.

Let's try to address this. Our first improvement to the vanilla **GPT-3.5** strategy is to include search results in the LM prompt, often called a *retrieve-then-read* approach.

In particular, we'll retrieve the most relevant passage to the question (according to the retriever) from the corpus and concatenate it into the prompt. This may help GPT-3.5 answer more factually.

To do this, we'll need to modify the `Template`. In particular, we'll define a `qa_template_with_passages` template, which will include a `context` field in addition to a question and an answer.

In [10]:
Context = dsp.Type(
    prefix="Context:\n",
    desc="${one or more sources, some of which may contain relevant content or clues}",
    format=dsp.passages2text
)

qa_template_with_passages = dsp.Template(
    instructions=qa_template.instructions,
    context=Context(), question=Question(), answer=Answer()
)

Now let's modify the program. We'll call `dsp.retrieve`, which we hooked earlier in this notebook to a **ColBERTv2** retriever serving a Wikipedia (Dec 2018) index.

In [11]:
def retrieve_then_read_QA(question: str) -> str:
    demos = dsp.sample(train, k=7)
    passages = dsp.retrieve(question, k=1)
    
    example = dsp.Example(question=question, context=passages, demos=demos)
    example, completions = dsp.generate(qa_template_with_passages)(example, stage='qa')

    return completions.answer

Let's invoke this on a sample question and let's inspect the last call to the LM to learn more about the behavior of the program.

In [12]:
retrieve_then_read_QA(dev[0].question), lm.inspect_history(n=1)





Answer questions with short factoid answers.

---

Question: Which award did the first book of Gary Zukav receive?
Final Answer: U.S. National Book Award

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Final Answer: Foxcatcher

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Final Answer: Kevin Greutert

Question: Who produced the album that included a re-recording of "Lithium"?
Final Answer: Butch Vig

Question: What city was the victim of Joseph Druces working in?
Final Answer: Boston, Massachusetts

Question: In what year was the star of To Hell and Back born?
Final Answer: 1925

---

Follow the following format.

Context:
${one or more sources, some of which may contain relevant content or clues}

Question: ${the question to be answered}

Final Answer: ${a short factoid answer, often between 1 and 5 words}

---

Context:
«Julia Peterkin | Julia Peterkin Julia Peterkin (October 31, 1880 – August 10,

('E. L. Doctorow', None)

Now, let's evaluate this on the dev examples.

In [13]:
evaluate(retrieve_then_read_QA, dev)

100%|██████████| 13/13 [00:00<00:00, 208.59it/s]

Answered 5 / 13 correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Daytona Beach, Florida",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1856,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],Studs Terkel,❌
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Manchester, England",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']",5 August 1862,❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Dundalk,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Jet Propulsion,❌


### Program 3: Retrieve-then-Read w/ Self-Consistency

It's clear that retrieval has the capacity to help **GPT-3.5** answer a larger number of questions more factually. However, these questions are too complicated for a single retrieved passage to suffice.

In this program, we try to make some improvements to the pipeline from **Program 2**.

In particular:

- We will include 5 passages (instead of a single passage) into the prompt.
- We will ask GPT-3.5 to generate a Chain-of-Thought (CoT) rationale to more effectively extract the answer from the passages.
- We will use Self-Consistency (SC) to marginalize this prediction across many chains of thought.

In [14]:
Rationale = dsp.Type(
    prefix="Rationale: Let's think step by step.",
    desc="${a step-by-step deduction that identifies the correct response, which will be provided below}"
)

qa_template_with_CoT = dsp.Template(
    instructions=qa_template.instructions,
    context=Context(), question=Question(), rationale=Rationale(), answer=Answer()
)

The snippet above updates our template to include a `rationale` field.

Below, we define the new program `retrieve_then_read_QA_v2`. In it, we define a new DSP _transformation_ `QA_predict`, which we will re-use in multiple programs.

The transformation `QA_predict` takes an `Example` (which has a question, context, and demonstrations) and generates an answer. It can be configured to use self-consistency (`sc=True`) or not.

In [15]:
@dsp.transformation
def QA_predict(example: dsp.Example, sc=True):
    if sc:
        example, completions = dsp.generate(qa_template_with_CoT, n=20, temperature=0.7)(example, stage='qa')
        completions = dsp.majority(completions)
    else:
        example, completions = dsp.generate(qa_template_with_CoT)(example, stage='qa')
    
    return example.copy(answer=completions.answer)

def retrieve_then_read_QA_v2(question: str) -> str:
    demos = dsp.sample(train, k=7)
    passages = dsp.retrieve(question, k=5)
    example = dsp.Example(question=question, context=passages, demos=demos)
    
    return QA_predict(example).answer

Let's inspect an example.

In [16]:
retrieve_then_read_QA_v2(dev[2].question), lm.inspect_history(n=1)





Answer questions with short factoid answers.

---

Question: Which award did the first book of Gary Zukav receive?
Final Answer: U.S. National Book Award

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Final Answer: Foxcatcher

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Final Answer: Kevin Greutert

Question: Who produced the album that included a re-recording of "Lithium"?
Final Answer: Butch Vig

Question: What city was the victim of Joseph Druces working in?
Final Answer: Boston, Massachusetts

Question: In what year was the star of To Hell and Back born?
Final Answer: 1925

---

Follow the following format.

Context:
${one or more sources, some of which may contain relevant content or clues}

Question: ${the question to be answered}

Rationale: Let's think step by step. ${a step-by-step deduction that identifies the correct response, which will be provided below}

Final Answer: ${a short facto

('Ocala, Florida', None)

And let's run this on the other dev examples.

In [17]:
evaluate(retrieve_then_read_QA_v2, dev)

100%|██████████| 13/13 [00:00<00:00, 16.43it/s]

Answered 6 / 13 correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",Julia Peterkin,❌
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Ocala, Florida",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1824,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","September 27, 1974",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Dundalk,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Aerospace engineering,❌


### Program 4: Multi-Hop Retrieval

From these simple programs, it becomes clear that a single search query is not enough for this task. This can be seen when trying to find the birth city of the writer of "Right Back At It Again". A search query identifies the author correctly as "Jeremy McKinnon", but it doesn't mention when he was born.

The standard approach for this challenge in the retrieval-augmented NLP literature is to build multi-hop search systems, like GoldEn (Qi et al., 2019), IRRR (Qi et al., 2021) and Baleen (Khattab et al., 2021). These systems read the retrieved results and then generate additional queries to gather additional information if necessary.


With **DSP**, we can easily simulate such systems in a few lines of code. Let's begin by defining two new templates.

First, in `rewrite_template`, we will re-write the input question into a simple search query. This will serve as our "first hop" query.

In [18]:
SearchRationale = dsp.Type(
    prefix="Rationale: Let's think step by step. To answer this question, we first need to find out",
    desc="${missing information}"
)

SearchQuery = dsp.Type(
    prefix="Search Query:",
    desc="${a simple question for seeking the missing or required information}"
)

rewrite_template = dsp.Template(
    instructions="Write a search query that will help answer a complex question.",
    question=Question(), rationale=SearchRationale(), query=SearchQuery()
)

Next, in `hop_template`, we will use the retrieved information from earlier hops to generate additional search queries for missing information.

In [19]:
CondenseRationale = dsp.Type(
    prefix="Rationale: Let's think step by step. We now know that",
    desc="${information from the context that may provide useful clues for the question}"
)

hop_template = dsp.Template(
    instructions=rewrite_template.instructions,
    context=Context(), question=Question(), rationale=CondenseRationale(), query=SearchQuery()
)

Let's build our `multihop_QA_v1` program. The hallmark of this program is the search transformation called `multihop_search_v1`.

This will largely simulate the IRRR system. In each hop, we will retrieve `k` (e.g., `k=2`) passages and concatenate them to the context retrieved from earlier hops to form a `context` chain.

For simplicity, we will repeat this for a total of `max_hops` hops (e.g., `max_hops=2`). It's easy to add a stopping criteria in which the LM is asked whether more hops are required.

In [20]:
from dsp.utils import deduplicate

@dsp.transformation
def multihop_search_v1(example: dsp.Example, max_hops=2, k=2) -> dsp.Example:
    example.context = []
    
    for hop in range(max_hops):
        # Generate a query based
        template = rewrite_template if hop == 0 else hop_template
        example, completions = dsp.generate(template)(example, stage=f'h{hop}')

        # Retrieve k results based on the query generated
        passages = dsp.retrieve(completions.query, k=k)

        # Update the context by concatenating old and new passages
        example.context = deduplicate(example.context + passages)

    return example


def multihop_QA_v1(question: str) -> str:
    demos = dsp.sample(train, k=7)
    x = dsp.Example(question=question, demos=demos)
    
    x = multihop_search_v1(x)
    x = QA_predict(x, sc=False)

    return x.answer

Let's run this on one example and inspect the last 3 calls to the LM (i.e., search query 1, search query 2, and QA).

In [21]:
multihop_QA_v1(dev[2].question), lm.inspect_history(n=3)





Write a search query that will help answer a complex question.

---

Follow the following format.

Question: ${the question to be answered}
Rationale: Let's think step by step. To answer this question, we first need to find out ${missing information}
Search Query: ${a simple question for seeking the missing or required information}

---

Question: Right Back At It Again contains lyrics co-written by the singer born in what city?
Rationale: Let's think step by step. To answer this question, we first need to find out the name of the singer
Search Query: Who sings Right Back At It Again







Write a search query that will help answer a complex question.

---

Follow the following format.

Context:
${one or more sources, some of which may contain relevant content or clues}

Question: ${the question to be answered}

Rationale: Let's think step by step. We now know that ${information from the context that may provide useful clues for the question}

Search Query: ${a simple question for

('Gainesville, Florida', None)

Now, let's run this on the remaining dev examples.

In [22]:
evaluate(multihop_QA_v1, dev)

100%|██████████| 13/13 [00:00<00:00, 83.91it/s]

Answered 7 / 13 correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Gainesville, Florida",✔️
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1947,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Manchester, England",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']",1965,❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",River Foyle,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Aerospace engineering,❌


### Program 5: Multi-Hop Condensed Retrieval w/ Automatic Demos and Query Fusion

Through Program 4, we've begun to explore some of the power of the DSP abstraction. However, if you look closely, you will see a few downsides of the previous approach:

1. The search transformations invoke the LM without any demonstrations in the prompt. That is because we only have training data for the question–answer pairs and not for the intermediate labels (e.g., search queries).
2. The QA prompt uses passages (`context`) and a Chain-of-Thought (`rationale`) for the question to be answered. However, the training demonstrations include neither context nor CoT because they aren't available in our labels.
3. The search transformations commit to a single query per hop, which may single out an unproductive chain of passages and hence fail to uncover relevant information.

We address these problems automatically in the program below.

In it, we begin by defining `multihop_demonstrate` (which uses `multihop_attempt`) to automatically **annotate** demonstrations for the complex multi-hop pipeline. These demonstrations will be provided to the LM when it's invoked for each transformation.

In [23]:
@dsp.transformation
def multihop_attempt(d: dsp.Example) -> dsp.Example:
    # Prepare unaugmented demonstrations for the example.
    x = dsp.Example(question=d.question, demos=dsp.all_but(train, d))
    
    # Search. And skip examples where search fails.
    # Annotate demonstrations for multihop_search_v2 with the simpler multihop_search_v1 pipeline.
    x = multihop_search_v1(x)
    if not dsp.passage_match(x.context, d.answer): return None
    
    # Predict. And skip examples where predict fails.
    x = QA_predict(x, sc=False)
    if not dsp.answer_match(x.answer, d.answer): return None
    
    return d.copy(**x)

@dsp.transformation
def multihop_demonstrate(x: dsp.Example) -> dsp.Example:
    demos = dsp.sample(train, k=7)
    x.demos = dsp.annotate(multihop_attempt)(demos, k=3, return_all=True)
    return x

We now implement `multihop_search_v2` as part of `multihop_QA_v2`. In addition to the changes mentioned earlier, this program simulates the Baleen system (Khattab et al., 2021)  in a few lines of code. In each retrieval hop (after the very first hop), the summary of the previous hop(s) is included in the prompt. This allows us to efficiently read a larger number of passages from the current hop.

In [24]:
@dsp.transformation
def multihop_search_v2(example: dsp.Example, max_hops=2, k=5) -> dsp.Example:
    example.context = []

    for hop in range(max_hops):
        # Generate queries
        template = rewrite_template if hop == 0 else hop_template
        example, completions = dsp.generate(template, n=10, temperature=0.7)(example, stage=f'h{hop}')
        
        # Collect the queries and search with result fusion
        queries = [c.query for c in completions] + [example.question]
        example.context = dsp.retrieveEnsemble(queries, k=k)

        # Arrange the passages for the next hop
        if hop > 0:
            example.context = [completions[0].rationale] + example.context
    
    return example

def multihop_QA_v2(question: str) -> str:
    x = dsp.Example(question=question)
    x = multihop_demonstrate(x)
    x = multihop_search_v2(x)
    x = QA_predict(x)
    return x.answer

In [25]:
multihop_QA_v2(dev[3].question), lm.inspect_history(n=3)





Write a search query that will help answer a complex question.

---

Follow the following format.

Question: ${the question to be answered}
Rationale: Let's think step by step. To answer this question, we first need to find out ${missing information}
Search Query: ${a simple question for seeking the missing or required information}

---

Question: Which award did the first book of Gary Zukav receive?
Rationale: Let's think step by step. To answer this question, we first need to find out the name of the first book of Gary Zukav
Search Query: "Gary Zukav" first book

---

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Rationale: Let's think step by step. To answer this question, we first need to find out who the heir to the Du Pont family fortune is.
Search Query: "Heir to the Du Pont family fortune"

---

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Rationale: Let's think step by step. To answer this

('1828', None)

In [26]:
evaluate(multihop_QA_v2, dev)

100%|██████████| 13/13 [00:01<00:00,  8.50it/s]

Answered 12 / 13 correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Gainesville, Florida",✔️
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1828,✔️
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']",Cheshire,✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']",1958,❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Castletown River,✔️
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Aeronautics,✔️


### Additional Programs...

While Program 5 is undeniably better than, say, Program 1, it's far from perfect.

Many different ideas can easily be implemented to make it more general or more powerful. Refer to the DSP paper for inspiration!

One of your first improvements to Programs 4 and 5 would be to allow it to do more than two hops and to automatically stop when additional hops are not required.

**Hint:** This is pretty easy to do. You can start with Program 4 and modify `hop_template` so its `SearchQuery` produces the string `N/A` when additional hops are not required.

In the `multihop_search_v1` function, you can then break from the loop — i.e., `if completions.query == 'N/A': break` whenever this happens. How will you apply this to Program 5?

```
SearchQueryWithStopping = dsp.Type(
    prefix="Search Query:",
    desc="${a simple question for seeking the missing or required information --- say N/A if the context above contains all of the required information}"
)
```

In [27]:
# Program 4 with max_hops=4 and automatic stopping.